# Collision Severity Prediction with Machine Learning Algorithmes - Ramin Ferdos

## Introduction

In this notebook, we are going to analyze histoical data which about accidents and its severities, then with using machine learning algorithmes predicting the severity of newly entered collisions. We are approaching this problem with CRISP-DM Methodologies and further you can find related steps with sections.

---

## First Step - Business Understanding

There are plenty of severity related to coliisons, for example, in a collision severities may be fatality, serious injury, injury, prop damage and etc. but we are reducing this types to two situation: 1. Property only damage collisions and 2. Injury collision.
With appropriate data we can extract valuable knowledge from this prediction we are going to just name a few, for instance, we can understand the probability of injuries in types of collisions for example, rear end collisions result injuries or sidesweeps? also we can mark dangerous spots which the occurance of injury collisions in them are siginifcant, another insight which we can extract is the occurance of collisions especially injury ones in which weathers are more.
so, our problem is predicting severity type of collions based on some features. we will talk about features further.

---

## Second Step - Analytical Approach

Based on the definition of problem, we are going to classify collisions, to two categories. obviously, we need to use classification algorithmes for this problem, there are multiple classifications algorithmes:
- K Nearest Neighbor
- Decision Tree
- Logistic Regression
- Support Vector Machine
- and etc.

It seems like Logistic Regression is a good approach for this problem since our dependent variable (Y) can be just two states (severity of collison type) and also this algorithmes will give us the probablity of each state (i.e for a specific sitations we are predicting that this collison severity will be 89% property damage and 11% injury.)

------

## Third Step - Data Requirements

As we discussed previously, we need data in order to predict our dependent variable. Our data should meet some requirements and sufficient features to make this prediction happen. Fortunately we have a rich data set containing required features and labels to predict. We will use effective features of this dataset as independent variable (x) and make this prediction happening.

----

## Fourth Step - Data Collections

In this step data scientists try to gather their required data as initial dataset to process it and clean it and predict based on it. Fortunately IBM hosted our data set which you can download using this link : [Downloading Dataset](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv) 

Also this Dataset is available at this project's GitHub Repository.
In this section we are going to load our data and explore it for a little bit.

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [3]:
collisions = pd.read_csv('Data-Collisions.csv')
print('Data is successfully read.')

Data is successfully read.


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
collisions.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [9]:
print(f'Our Initial Dataset contains {collisions.shape[0]} records and {collisions.shape[1]} features.')

Our Initial Dataset contains 194673 records and 38 features.


as you can see above we have read the data successfully, checked its heads and discovered number of total records and features.

----

## Fifth Step - Data Preparations

This section is one the most important steps in our methodology. In this step we are going to prepare our data to process and prediction. this step takes most of projects times usually and should be taken with caution. Fortunately since our dataset is clear and processed this important step is not a sore thumb in our project. Regardless, we should go through this step carefully and completely.

Looking to our dataset, we will understand that our severity codes signifies this:
1. Property Damage Only Collision
2. Injury Collision

In [13]:
collisions['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

According to our dataset, out of 194673 records, 136485 records of collisions were 'Property damage only collisions' and 58188 were 'injury collisions'.
There are some feature which won't help us to predict and we can delete them, this features are like:
- REPORTNO
- STATUS
- INCKEY
- etc.


In [22]:
collisions.columns = collisions.columns.str.lower()
collisions.drop(columns=['status','inckey','coldetkey','reportno','intkey','location','exceptrsncode','exceptrsndesc','seglanekey','crosswalkkey','severitycode.1'], inplace=True)

,severitycode,objectid,addrtype,severitycode.1,severitydesc,collisiontype,personcount,pedcount,pedcylcount,vehcount,...,underinfl,weather,roadcond,lightcond,pedrownotgrnt,sdotcolnum,speeding,st_colcode,st_coldesc,hitparkedcar
0,2,1,Intersection,2,Injury Collision,Angles,2,0,0,2,...,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,N
1,1,2,Block,1,Property Damage Only Collision,Sideswipe,2,0,0,2,...,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,N
2,1,3,Block,1,Property Damage Only Collision,Parked Car,4,0,0,3,...,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,N
3,1,4,Block,1,Property Damage Only Collision,Other,3,0,0,3,...,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,N
4,2,5,Intersection,2,Injury Collision,Angles,2,0,0,2,...,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,N


In [31]:
collisions.drop(columns=['sdotcolnum'], inplace=True)

In [32]:
collisions.head()

,severitycode,objectid,addrtype,severitydesc,collisiontype,personcount,pedcount,pedcylcount,vehcount,incdate,...,inattentionind,underinfl,weather,roadcond,lightcond,pedrownotgrnt,speeding,st_colcode,st_coldesc,hitparkedcar
0,2,1,Intersection,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,...,NaN,N,Overcast,Wet,Daylight,NaN,0,10,Entering at angle,N
1,1,2,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,...,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,0,11,From same direction - both going straight - bo...,N
2,1,3,Block,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,...,NaN,0,Overcast,Dry,Daylight,NaN,0,32,One parked--one moving,N
3,1,4,Block,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,...,NaN,N,Clear,Dry,Daylight,NaN,0,23,From same direction - all others,N
4,2,5,Intersection,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,...,NaN,0,Raining,Wet,Daylight,NaN,0,10,Entering at angle,N


now we have to make our dataset cohesive. It means that everywhere we have 'No,no,n,0,null,NaN' should be changed into 0 and everywhere we have 'Yes,y,yes,1' should be changed into 1.

In [78]:
test = collisions
test['speeding'] = test['speeding'].replace(np.nan, 0)
test['speeding'] = test['speeding'].replace('Y', 1)

test['pedrownotgrnt'] = test['pedrownotgrnt'].replace(np.nan, 0)
test['pedrownotgrnt'] = test['pedrownotgrnt'].replace('Y', 1)

test['underinfl'] = test['underinfl'].replace('N', 0)
test['underinfl'] = test['underinfl'].replace('0', 0)
test['underinfl'] = test['underinfl'].replace(np.nan, 0)
test['underinfl'] = test['underinfl'].replace('Y', 1)
test['underinfl'] = test['underinfl'].replace('1', 1)
test['underinfl'] = test['underinfl'].astype(int)

test['inattentionind'] = test['inattentionind'].replace(np.nan, 0)
test['inattentionind'] = test['inattentionind'].replace('Y', 1)

test['hitparkedcar'] = test['hitparkedcar'].replace('N', 0)
test['hitparkedcar'] = test['hitparkedcar'].replace('Y', 1)


In [75]:
collisions = test
collisions.head()

,severitycode,objectid,addrtype,severitydesc,collisiontype,personcount,pedcount,pedcylcount,vehcount,incdate,incdttm,junctiontype,sdot_colcode,sdot_coldesc,inattentionind,underinfl,weather,roadcond,lightcond,pedrownotgrnt,speeding,st_colcode,st_coldesc,hitparkedcar
0,2,1,Intersection,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 14:54,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,0,Overcast,Wet,Daylight,0,0,10,Entering at angle,0
1,1,2,Block,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 18:55,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",0,0,Raining,Wet,Dark - Street Lights On,0,0,11,From same direction - both going straight - bo...,0
2,1,3,Block,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",0,0,Overcast,Dry,Daylight,0,0,32,One parked--one moving,0
3,1,4,Block,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,0,Clear,Dry,Daylight,0,0,23,From same direction - all others,0
4,2,5,Intersection,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",0,0,Raining,Wet,Daylight,0,0,10,Entering at angle,0


next thing we should address is our categorical variables.

In [ ]:
collisions.